In [1]:
import os
os.getcwd()

'/Users/praveen/MSDS/MachineLearning/KaggleChallenges/Challenge2/Data'

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd

# Assume df is preprocessed and cleaned
df = pd.read_csv("./training_data.csv", low_memory=False)

# Convert problematic columns
for col in ['governor_contribution_ratio', 'senate_contribution_ratio', 'us_senate_contribution_ratio']:
    df[col] = pd.to_numeric(df[col], errors='coerce')


# Drop high cardinality or non-informative
df = df.drop(columns=['zip_code'])
df = df.dropna()

# Binary label — you can tune this threshold
df['won'] = (df['winner_ratio'] > 0.0).astype(int)

df.head()

,general_sector,city,specific_sector,state,contributor_type,winner_ratio,candidacy_count,candidacy_democratic_count,candidacy_republican_count,contribution_count,...,house_and_assembly_contribution_ratio,politician_challenger_ratio,politician_democratic_ratio,politician_incumbency_ratio,politician_open_pos_ratio,politician_republican_ratio,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio,won
0,Retired,MISSOULA,Retired,MT,Individual,1.0,4,4,0,6,...,0.000000,0.0,1.0,0.5,0.5,0.0,0.000000,0.0,0.0,1
1,Retired,CLEVELAND HTS,Retired,OH,Individual,0.5,2,2,0,2,...,0.216202,0.5,1.0,0.5,0.0,0.0,0.000000,0.0,0.0,1
2,Public Sector Unions,NEWARK,Teachers unions,NJ,Non-Individual,1.0,4,4,0,4,...,3.257082,0.0,1.0,1.0,0.0,0.0,-2.257082,0.0,0.0,1
3,Retired,BOZEMAN,Retired,MT,Individual,1.0,4,4,0,4,...,0.200000,0.0,1.0,0.5,0.5,0.0,0.200000,0.0,0.0,1
4,No Employer Listed or Found,ORANGE PARK,No employer listed or discovered,FL,Individual,0.0,4,4,0,6,...,0.000000,0.5,1.0,0.0,0.5,0.0,0.000000,0.0,0.0,0


In [9]:
df.columns

Index(['general_sector', 'city', 'specific_sector', 'state',
       'contributor_type', 'winner_ratio', 'candidacy_count',
       'candidacy_democratic_count', 'candidacy_republican_count',
       'contribution_count', 'contribution_democratic_count',
       'contribution_republican_count', 'politician_challenger_count',
       'politician_count', 'politician_democratic_count',
       'politician_incumbency_count', 'politician_open_pos_count',
       'politician_republican_count', 'contribution_democratic_sum_2010_usd',
       'contribution_republican_sum_2010_usd', 'contribution_sum_2010_usd',
       'governor_contributions_sum_2010_usd',
       'house_and_assembly_contributions_sum_2010_usd',
       'senate_contributions_sum_2010_usd',
       'us_house_contributions_sum_2010_usd',
       'us_senate_contributions_sum_2010_usd', 'candidacy_democratic_ratio',
       'candidacy_republican_ratio', 'contribution_democratic_count_ratio',
       'contribution_republican_count_ratio', 'govern

In [1]:



# One-hot encode categorical vars
cat_cols = ['general_sector', 'city', 'specific_sector', 'state', 'contributor_type']
df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)

# Split
X = df_encoded.drop(columns=['winner_ratio', 'won'])
y_binary = df_encoded['won']
y_continuous = df_encoded['winner_ratio']

X_train, X_test, y_train_bin, y_test_bin, y_train_ratio, y_test_ratio = train_test_split(
    X, y_binary, y_continuous, test_size=0.2, random_state=42
)

In [3]:
df_encoded.shape

(171192, 9654)

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import joblib

# Initialize model with warm_start
clf = RandomForestClassifier(n_estimators=1, warm_start=True, random_state=42)

best_mse = float("inf")
best_model = None

mse_progress = []

# Build one tree per iteration
for i in tqdm(range(1, 101), desc="Training Progress"):
    clf.n_estimators = i
    clf.fit(X_train, y_train_bin)

    # Predict probabilities for class 1
    y_proba = clf.predict_proba(X_test)[:, 1]

    # Compare with actual winner_ratio
    mse = mean_squared_error(y_test_ratio, y_proba)
    mse_progress.append(mse)

    # Print progress (optional)
    print(f"Trees: {i}, MSE: {mse:.5f}")

    # Save best model
    if mse < best_mse:
        best_mse = mse
        best_model = clf
        joblib.dump(best_model, f"best_rf_model_{i}_trees.pkl")

print(f"\n✅ Best MSE: {best_mse:.5f}")


Training Progress:   1%|          | 1/100 [00:14<24:02, 14.58s/it]

Trees: 1, MSE: 0.21215


Training Progress:   2%|▏         | 2/100 [00:23<17:54, 10.96s/it]

Trees: 2, MSE: 0.16650


Training Progress:   3%|▎         | 3/100 [00:31<15:38,  9.68s/it]

...

Trees: 98, MSE: 0.12646


Training Progress:  99%|█████████▉| 99/100 [17:37<00:06,  6.48s/it]

Trees: 99, MSE: 0.12643


Training Progress: 100%|██████████| 100/100 [17:43<00:00, 10.64s/it]

Trees: 100, MSE: 0.12644

✅ Best MSE: 0.12643


In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1, 101), mse_progress, marker='o')
plt.xlabel('Number of Trees')
plt.ylabel('MSE')
plt.title('MSE vs Number of Trees')
plt.grid(True)
plt.show()
